<a href="https://colab.research.google.com/github/Brevex/US-Used-Cars-Data-Analysis/blob/main/US_Used_Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask
!pip install opendatasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import opendatasets as od
import matplotlib.pyplot as plt

from dask import dataframe as dd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
od.download("https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset")

In [ ]:
dask_df = dd.read_csv('/content/us-used-cars-dataset/used_cars_data.csv', blocksize="64MB",
       usecols=[
          'engine_cylinders',
          'fuel_tank_volume',
          'horsepower',
          'make_name',
          'mileage',
          'model_name',
          'power',
          'price',
          'torque',
          'transmission',
          'year'
       ])

df = dask_df.compute()
df

In [ ]:
df.info()

In [ ]:
count = 0
nan_col = []
col_percent = []

for col in df.columns:
    nan_qntt = df[col].isna().sum()

    if nan_qntt > 0:

        percent = nan_qntt/df.shape[0]*100
        print('A coluna {} tem {}% NaN'.format(col,[percent]))
        count += 1
        nan_col += [col]
        col_percent += [(col,percent)]

print(count)
print(nan_col)

In [ ]:
df.isna().sum()

In [ ]:
ignored_nan_df = df.dropna()
ignored_nan_df

In [ ]:
df_no_mileage = df.drop(axis=1, labels='mileage')
ignoring_nan_and_mileage_df = df_no_mileage.dropna()
ignoring_nan_and_mileage_df
ignoring_nan_and_mileage_df.info()

In [ ]:
df_tratando = ignoring_nan_and_mileage_df.drop_duplicates()

# Removing strings from data
df_tratando['fuel_tank_volume'] = df_tratando['fuel_tank_volume'].replace('--', np.nan)
df_tratando['fuel_tank_volume'] = df_tratando['fuel_tank_volume'].str.replace(' gal', '').astype(float)

df_tratando = df_tratando.drop('power', axis = 1)
df_tratando = df_tratando.drop('torque', axis = 1)

df_final = df_tratando.dropna()

In [ ]:
df_final.head(10)

# Importance score for each variable

In [ ]:
# Defining the input variables (X) and the target variable (y)
X = df_final.drop("price", axis=1)
y = df_final["price"]

# Encoding categorical variables using LabelEncoder
categorical_cols = ["engine_cylinders", "make_name", "model_name", "transmission"]

for col in categorical_cols:

    encoder = LabelEncoder()
    X[col] = encoder.fit_transform(X[col])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating an instance of the Random Forest model
model = RandomForestRegressor()

# Training the model with the training data
model.fit(X_train, y_train)

# Making predictions with the test set
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Printing the evaluation metrics
print(f"R-squared: {r2}")
print()
print("Importance of each feature: ")
print()

# Printing the importance of features
importances = model.feature_importances_
columns = X.columns

for col, importance in zip(columns, importances):
    print(f"{col}: {importance}")

#Horsepower x Engine Chart

In [ ]:
# Grouping the data by 'engine_cylinders' and calculate the average of 'horsepower'
mean_horsepower = df.groupby('engine_cylinders')['horsepower'].mean()

# Increasing the chart width
plt.figure(figsize=(12, 8))

# Creating a bar chart with average 'horsepower' by type of 'engine_cylinders'
mean_horsepower.plot(kind='bar')

# Customizing the chart
plt.title("Average Horsepower by Type of Engine Cylinders")
plt.xlabel("Engine Cylinders")
plt.ylabel("Average Horsepower")

plt.show()

#Average Horsepower score per engine

In [ ]:
# Transforming categorical variables into numerical ones
label_encoder = LabelEncoder()

df_final['engine_cylinders_encoded'] = label_encoder.fit_transform(df_final['engine_cylinders'])
df_final['horsepower'] = pd.to_numeric(df_final['horsepower'])

# Calculating the average horsepower for each engine cylinders
grouped_data = df_final.groupby('engine_cylinders_encoded')['horsepower'].mean().reset_index()

# Normalizing the average of 'horsepower' between 0 and 1
min_horsepower = grouped_data['horsepower'].min()
max_horsepower = grouped_data['horsepower'].max()
grouped_data['score'] = (grouped_data['horsepower'] - min_horsepower) / (max_horsepower - min_horsepower)

# Sorting the results by score in ascending order
grouped_data = grouped_data.sort_values(by='score')

# Displaying the score for each engine type in ascending order
engine_cylinders_names = label_encoder.inverse_transform(grouped_data['engine_cylinders_encoded'])

for engine_cylinders, score in zip(engine_cylinders_names, grouped_data['score']):

    print(f"Engine Cylinders: {engine_cylinders}")
    print(f"Score: {score}\n")

#Prediction model with Engine x Horsepower score

In [ ]:
# Transforming categorical variables into numerical ones
label_encoder = LabelEncoder()

df_final['engine_cylinders_encoded'] = label_encoder.fit_transform(df_final['engine_cylinders'])
df_final['horsepower'] = pd.to_numeric(df_final['horsepower'])

# Separating the features (X) and the target (y)
X = df_final[['engine_cylinders_encoded']]
y = df_final['horsepower']

# Creating and training the random forest classification model
model = RandomForestClassifier()
model.fit(X, y)

# Getting the complete list of engine types
engine_cylinders_names = df_final['engine_cylinders'].unique()

# Making predictions for all engine types
predictions = model.predict(label_encoder.transform(engine_cylinders_names).reshape(-1, 1))

# Creating a dataframe with the results
results_df = pd.DataFrame({'Engine Cylinders': engine_cylinders_names, 'Predicted Horsepower': predictions})
results_df = results_df.sort_values(by='Predicted Horsepower')


print(results_df)